# Instance-Based Learning: K-Nearest Neighbours(kNN)

## 1. Data Preprocessing

In [80]:
#importing the libraries needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#scikit-learn library - not complete

In [81]:
train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')

data transformation of the categorical attrbutes to numeric

In [82]:
#transformation 
from sklearn.preprocessing import LabelEncoder

# Select the categorical columns to encode
cat_columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country"]

# Encode categorical columns using Label Encoding for data_train
le = LabelEncoder()
for col in cat_columns:
    train[col] = le.fit_transform(train[col])

# Encode categorical columns using Label Encoding for data_test
le = LabelEncoder()
for col in cat_columns:
    test[col] = le.fit_transform(test[col])


# Print the first 5 rows of the transformed dataset
#print(train.head())
#print(test.head())

In [83]:

#train
x_train_trans = train.drop(columns = ['income'])
y_train = train['income']

#test
x_test_trans = test.drop(columns =['income'])
y_test = test['income']

print(x_train_trans)
#print(y_train_trans)
#print(x_test_trans)


       age  workclass  fnlwgt  education  educational_num  marital-status  \
0       39          6   77516          9               13               4   
1       50          5   83311          9               13               2   
2       38          3  215646         11                9               0   
3       53          3  234721          1                7               2   
4       28          3  338409          9               13               2   
...    ...        ...     ...        ...              ...             ...   
32556   27          3  257302          7               12               2   
32557   40          3  154374         11                9               2   
32558   58          3  151910         11                9               6   
32559   22          3  201490         11                9               4   
32560   52          4  287927         11                9               2   

       occupation  relationship  race  gender  capital-gain  capital-loss  

Standard Scaling


StandardScaler is being used to standardize the data by removing the mean and scaling to unit variance. This is commn preprocessing step in machine learning to ensure that all features are on the same scale, which can improve the performance and accuracy of many algorithm

In [84]:
#Standard Scaling

from sklearn import preprocessing

x_train = preprocessing.StandardScaler().fit(x_train_trans).transform(x_train_trans.astype(float))
#print(x_train)

x_test = preprocessing.StandardScaler().fit(x_test_trans).transform(x_test_trans.astype(float))
print(x_test)

[[-0.99412926 -0.09872448  0.35347399 ... -0.21806206 -0.03143184
   0.25771223]
 [-0.05541716 -0.09872448 -0.94239062 ... -0.21806206  0.7699177
   0.25771223]
 [-0.77750339 -1.88674668  1.39544986 ... -0.21806206 -0.03143184
   0.25771223]
 ...
 [ 0.37783458 -0.09872448 -0.99842039 ... -0.21806206 -0.03143184
   0.25771223]
 [-0.27204303  0.79528662 -0.0689392  ... -0.21806206  1.57126723
   0.25771223]
 [        nan  4.37133104         nan ...         nan         nan
   0.77181055]]


## 2. Using default kNN

In [85]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


#base model
default_knn = KNeighborsClassifier()
default_knn.fit(x_train, y_train)

#predict
y_pred=default_knn.predict(x_test)

#Summarize Result
#precision,recall,f1-score,support, accuracy, macro avg, weighted avg
print(classification_report(y_test,y_pred))
#ROC score
print(roc_auc_score(y_test, y_pred))
#confusion matrix
print(confusion_matrix(y_test,y_pred))


ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

Doing a validation of performance and confusion matrix

## 4. Hyperparameter Tuning

Hyperparameter Tuning for K-neighbours, Weight and Distance Metric and validating its performance using accuracy, precision, recall and F1-score

This will be done by making a ranking of validation

-GridSearchCV 
with a sklearn.model_selection.GridSearchCV

-Training and Validation Split 10% 
    -goals on validation: how robust the validation - doesnt have much variance when you the validation again - (find the robust percentage-is it really 10%)

-Find whether training/validation split and k-folds is the best for kNN


### 4.1 Method: K-folds Cross Validation

K-folds Cross Validation is used as it uses all the data for training and validation.

In [68]:
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV


# Creating a parameter of the grid
param_grid = [{
    'n_neighbors': list(range(1, 30, 2)),
    'metric': ['euclidean', 'manhattan', 'cosine', 'minkowski'],
    'weights':['uniform','distance'],
    #'algorithm':['auto','ball_tree','kd-tree','brute'],
    'leaf_size' : list(range(1,50, 5))
}]
#is this how you put the hyperparameter range?


from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score, pos_label = 1, average = 'binary')

# Grid search model
knn_grid = KNeighborsClassifier()
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3, random_state=seed)
grid_search = GridSearchCV(estimator = knn_grid, param_grid = param_grid, cv =cv, verbose = 2, scoring = f1, n_jobs=-1)
#go back and play around with thi GridSearchCV parameter
#from youtube: grid_search = GridSearchCV(estimator=knn,param_grid=knn_param, n_jobs=1, cv=cv, scoring="accuracy", error_score=0)

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid=[{'leaf_size': [25, 30, 35, 40, 45],
                          'metric': ['euclidean', 'manhattan', 'cosine'],
                          'n_neighbors': [10, 15],
                          'weights': ['uniform', 'distance']}],
             scoring=make_scorer(f1_score, pos_label=1, average=binary),
             verbose=2)

This data is divided into k of 5 of equal parts (folds), and the model is trained and validated k times. 

In [69]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_leaf_size,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.110264,0.006097,12.821461,0.682926,25,euclidean,10,uniform,"{'leaf_size': 25, 'metric': 'euclidean', 'n_ne...",0.575769,0.591882,0.603128,0.618195,0.598165,0.597428,0.013887,56
1,0.112174,0.006768,12.787271,0.667127,25,euclidean,10,distance,"{'leaf_size': 25, 'metric': 'euclidean', 'n_ne...",0.602009,0.623891,0.632473,0.635707,0.629693,0.624755,0.012017,21
2,0.111524,0.007708,14.031569,0.601749,25,euclidean,15,uniform,"{'leaf_size': 25, 'metric': 'euclidean', 'n_ne...",0.601339,0.620979,0.628254,0.642412,0.626312,0.623859,0.013304,26
3,0.107700,0.010861,14.161390,0.624608,25,euclidean,15,distance,"{'leaf_size': 25, 'metric': 'euclidean', 'n_ne...",0.602519,0.622284,0.632414,0.640110,0.630518,0.625569,0.012844,16
4,0.108185,0.007996,17.313154,0.303038,25,manhattan,10,uniform,"{'leaf_size': 25, 'metric': 'manhattan', 'n_ne...",0.588192,0.609078,0.608916,0.619345,0.613043,0.607715,0.010470,46
5,0.117379,0.023993,17.198712,0.915965,25,manhattan,10,distance,"{'leaf_size': 25, 'metric': 'manhattan', 'n_ne...",0.612444,0.622526,0.633913,0.638879,0.630753,0.627703,0.009300,11
6,0.113706,0.013084,22.870516,2.403588,25,manhattan,15,uniform,"{'leaf_size': 25, 'metric': 'manhattan', 'n_ne...",0.614035,0.635766,0.641848,0.638238,0.635384,0.633054,0.009785,1
7,0.105166,0.013039,25.363310,0.707857,25,manhattan,15,distance,"{'leaf_size': 25, 'metric': 'manhattan', 'n_ne...",0.613820,0.634180,0.638108,0.642857,0.630924,0.631978,0.009916,6
8,0.013304,0.005794,9.350719,0.172086,25,cosine,10,uniform,"{'leaf_size': 25, 'metric': 'cosine', 'n_neigh...",0.570793,0.593601,0.607012,0.622842,0.605042,0.599858,0.017264,51
9,0.012401,0.002499,8.839355,0.418423,25,cosine,10,distance,"{'leaf_size': 25, 'metric': 'cosine', 'n_neigh...",0.591376,0.611486,0.616848,0.623866,0.622940,0.613303,0.011841,41


Finding what is the best score/optimal accuracy score

In [76]:
#Finding what is the best score/ optimal accuracy score
print("Best F1-score: ",grid_search.best_score_)

Best F1-score:  0.6330542712337375


Finding the hyperparameter to achieve this score

In [73]:
#Finding the hyperparameter to achieve this score
print("The list of hyperparmeter in order to achieve this best score: ", grid_search.best_params_)

The list of hyperparmeter in order to achieve this best score:  {'leaf_size': 25, 'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'uniform'}


## 5. Model with the best Hyperparameter

In [77]:
best_model = KNeighborsClassifier(n_neighbors=15, weights='uniform', leaf_size= 25, metric="manhattan",)

In [79]:
best_model.fit(x_train,y_train)
y_pred = best_model.predict(x_test)

#accuracy = best_model.score(x_train, y_train)
#print("Accuracy:", accuracy)

Accuracy: 0.8577132151960934


Doing a validation of performance and confusion matrix

## 6. Validation and Confusion Matrix

of the best model with the tuned hyperparameter

In [ ]:
#Summarize Result
#precision,recall,f1-score,support, accuracy, macro avg, weighted avg
print(classification_report(y_test,y_pred))
#ROC score
print(roc_auc_score(y_test, y_pred))
#confusion matrix
print(confusion_matrix(y_test,y_pred))

#again print the best hyperparameter tuned for the best model
print(grid_search.best_params_)

Finally, we can check if the accuracy had improve between the default_knn and when the hyperparameter is tuned for the best_model

## 7. Exporting the model

In [ ]:
import joblib
joblib.dump(tuned_model, 'kNN.pkl', compress=9)

#what is compress=9, do we need that